In [1]:
import pandas as pd
import re
import json
import math
from itertools import permutations, combinations
from IPython.display import clear_output
from time import sleep
from abc import ABC, abstractmethod, abstractproperty

from grid_builder import *
from grid import *
from tile import *
from tiles import *
from destructable import *
from unit import *
from command import *
from ability import *
from shell import *
from beam import *
from game import *
from executor import *
from dfs import *

In [2]:
class Perm:
    def __init__(self):
        self.options = ["Move", "Shoot"]
        self.ls = []
        self.perms = []
        self.recurse()
        
    def recurse(self):
        frontier = self.gen_frontier()
        if not frontier:
            self.perms.append(self.ls)
            print(self.ls)
            return
            
        for option in frontier:
            self.ls.append(option)
            self.recurse()
            self.ls.pop()
        
    def gen_frontier(self):
        frontier = []
        move_count = self.ls.count("Move")
        shoot_count = self.ls.count("Shoot")
        if move_count < 3:
            frontier.append("Move")
        
        if move_count > shoot_count and shoot_count < 3:
            frontier.append("Shoot")
            
        return frontier
        
perm = Perm()  

['Move', 'Move', 'Move', 'Shoot', 'Shoot', 'Shoot']
['Move', 'Move', 'Shoot', 'Move', 'Shoot', 'Shoot']
['Move', 'Move', 'Shoot', 'Shoot', 'Move', 'Shoot']
['Move', 'Shoot', 'Move', 'Move', 'Shoot', 'Shoot']
['Move', 'Shoot', 'Move', 'Shoot', 'Move', 'Shoot']


In [3]:
class Perm2:
    def __init__(self):
        self.shoots = ["Shoot1","Shoot2","Shoot3"]
        self.moves = ["Move1", "Move2", "Move3"]
        self.combos = []
        self.combo_1()
        self.combo_2()
        
    def __repr__(self):
        string = ""
        for i in self.combos:
            string += repr(i) +'\n'
        return string
            
    def last_move(self,tup):
        for move in self.moves:
            if move not in tup:
                return move,
            
            
    def generate(self, r=3):
        for shoot in permutations(shoots):
            for move in combinations(moves, r):
                yield move, shoot
                
    def combo_1(self):
        for move,shoot in self.generate(3):
            self.combos.append(move+shoot)
            
            
    def combo_2(self):
        for move,shoot in self.generate(2):
            move += self.last_move(move)
            self.combos.append((move[0],move[1],shoot[0],move[2],shoot[1],shoot[2]))
            self.combos.append((move[0],move[1],shoot[0],shoot[1],move[2],shoot[2]))
            self.combos.append((move[0],shoot[0],move[1],move[2],shoot[1],shoot[2]))
            self.combos.append((move[0],shoot[0],move[1],shoot[1],move[2],shoot[2]))
            
            
    def add_combos_3(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=3):
                self.combos.append(move+shoot)
                
                
    def add_combos_2(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=2):
                move += self.last_move(move)
                
            
            
    def stuff(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=2):
                print((x[0],x[1],y[0],y[1],last_elem(x),y[1]))

In [4]:
gb = GridBuilder()

gb.place( WaterTile(),      (0,4) )
gb.place( WaterTile(),      (0,5) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (0,6) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (0,7) )

gb.place( Destructable(CorporateTile(), GroundTile(), 1), (1,1) )
gb.place( ForestTile(),     (1,2) )
gb.place( Destructable(CivilianTile(), GroundTile(), 1),  (1,3) )
gb.place( Destructable(CivilianTile(), GroundTile(), 1),  (1,4) )
gb.place( WaterTile(),      (1,5) )
gb.place( WaterTile(),      (1,6) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (1,7) )

gb.place( ForestTile(),     (2,3) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (2,5) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (2,6) )
gb.place( WaterTile(),      (2,7) )

gb.place( Destructable(CivilianTile(), GroundTile(), 1),  (3,1) )
gb.place( WaterTile(),      (3,7) )

gb.place( Destructable(CivilianTile(), GroundTile(), 2),  (4,1) )
gb.place( Destructable(CivilianTile(), GroundTile(), 2),  (4,5) )
gb.place( Destructable(CivilianTile(), GroundTile(), 2),  (4,6) )

gb.place( ForestFireTile(), (5,1) )
gb.place( ForestFireTile(), (5,2) )
gb.place( SpawnTile(),      (5,4) )
gb.place( ForestTile(),     (5,7) )

gb.place( Destructable(MountainTile(), GroundTile(), 3),   (6,0) )
gb.place( SpawnTile(),      (6,2) )
gb.place( ForestFireTile(), (6,3) )
gb.place( ForestFireTile(), (6,4) )

gb.place( Destructable(MountainTile(), GroundTile(), 3),   (7,0) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (7,1) )
gb.place( ForestTile(),     (7,3) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (7,7) )

gb.place_on_tile( Mech("Siege Mech", 5, 4),     (3,2) )
gb.place_on_tile( Mech("Artillery Mech", 5, 1), (5,3) )
gb.place_on_tile( Mech("Boulder Mech", 5, 5, 4),   (7,3) )
gb.place_on_tile( Vek("Psy", 5, 2),             (4,2) )
gb.place_on_tile( Vek("Alpha Firefly", 6, 5, 2),   (4,4) )
gb.place_on_tile( Vek("Scarab", 3, 3),          (6,1) )
gb.place_on_tile( Vek("Firefly", 4, 4, 2),         (6,5) )
gb.place_on_tile( Unit("Boulder", max_health=1, health=1, moves=0), (7,5) )

grid = gb.to_grid()

for mech in grid.mechs:
    mech.add(Move(mech,grid))
    mech.add(Repair(mech))
    
for vek in grid.veks:
    vek.add(Move(vek, grid))

    
siege_mech     = grid.units['Siege Mech']
artillery_mech = grid.units['Artillery Mech']
boulder_mech   = grid.units['Boulder Mech']

firefly        = grid.units['Firefly']
alpha_firefly  = grid.units['Alpha Firefly']
scarab         = grid.units['Scarab']
psy            = grid.units['Psy']


siege_mech.add(Artillery(siege_mech, grid, ClusterShell, 2))
boulder_mech.add(Artillery(boulder_mech, grid, BoulderShell, 2))
artillery_mech.add(Artillery(artillery_mech, grid, RegularShell, 1))

firefly.add(Beam(firefly, grid, VekBeam, 1))
alpha_firefly.add(Beam(alpha_firefly, grid, VekBeam, 3))
scarab.add(Artillery(scarab, grid, VekShell, 1))

firefly.target = CommandDecorator(firefly, VekBeam(firefly, grid, Compass.NORTH, 1))
alpha_firefly.target = CommandDecorator(alpha_firefly, VekBeam(alpha_firefly, grid, Compass.NORTH, 3))
scarab.target = CommandDecorator(scarab, VekShell(scarab, grid, 1, (1,1)))
    
grid.show()

,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,🌲,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,🟢,🟢,🟢,🌲,🟢,🗻,🗻,💦
3,🟢,🏘️ ϟ,Siege Mech ♡♡♡♡,🟢,🟢,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,Psy ♡♡,🟢,Alpha Firefly ♡♡♡♡♡,🏘️ ϟϟ,🏘️ ϟϟ,🟢
5,🟢,🔥,🔥,Artillery Mech ♡,⬆️,🟢,🟢,🌲
6,🗻,Scarab ♡♡♡,⬆️,🔥,🔥,Firefly ♡♡♡♡,🟢,🟢
7,🗻,🗻,🟢,Boulder Mech ♡♡♡♡♡,🟢,Boulder ♡,🟢,🗻


In [5]:
dfs = DFS(grid, 5)

new score 34 != old score 33 at MOVE (7, 3) to (5, 5)_MOVE (3, 2) to (1, 2)_CLUSTER SHELL at (3, 2)_
[MOVE (7, 3) to (5, 5), MOVE (3, 2) to (3, 4), CLUSTER SHELL at (3, 2), BOULDER SHELL at (5, 1), MOVE (5, 3) to (7, 3), HEAL +1, REGULAR SHELL at (7, 1), MOVE (5, 3) to (3, 2), HEAL +1, REGULAR SHELL at (6, 2), MOVE (5, 3) to (3, 3), HEAL +1, MOVE (5, 3) to (7, 4), HEAL +1, REGULAR SHELL at (4, 4), REGULAR SHELL at (7, 1), MOVE (5, 3) to (2, 3), HEAL +1, MOVE (5, 3) to (7, 3), REGULAR SHELL at (7, 1), HEAL +1, BOULDER SHELL at (5, 1), BOULDER SHELL at (5, 1), HEAL +1, REGULAR SHELL at (7, 1), MOVE (5, 3) to (3, 3), HEAL +1, BOULDER SHELL at (5, 1), BOULDER SHELL at (5, 1), HEAL +1, MOVE (5, 3) to (3, 2), HEAL +1, BOULDER SHELL at (5, 1), REGULAR SHELL at (6, 2), BOULDER SHELL at (5, 1), BOULDER SHELL at (5, 1), HEAL +1, REGULAR SHELL at (6, 2), MOVE (5, 3) to (7, 4), REGULAR SHELL at (7, 1), HEAL +1, BOULDER SHELL at (5, 1), REGULAR SHELL at (4, 4), BOULDER SHELL at (5, 1), BOULDER SHEL

In [6]:
dfs.show()

56
MOVE (7, 3) to (5, 5)
BOULDER SHELL at (5, 1)
MOVE (3, 2) to (1, 2)
MOVE (5, 3) to (7, 3)
CLUSTER SHELL at (5, 2)
REGULAR SHELL at (7, 1)

56
MOVE (7, 3) to (5, 5)
BOULDER SHELL at (5, 1)
MOVE (3, 2) to (3, 2)
MOVE (5, 3) to (7, 3)
CLUSTER SHELL at (5, 2)
REGULAR SHELL at (7, 1)

46
MOVE (7, 3) to (5, 5)
BOULDER SHELL at (5, 1)
MOVE (3, 2) to (1, 2)
CLUSTER SHELL at (6, 2)
MOVE (5, 3) to (6, 1)
REGULAR SHELL at (6, 6)

46
MOVE (7, 3) to (5, 5)
BOULDER SHELL at (5, 1)
MOVE (3, 2) to (3, 2)
CLUSTER SHELL at (6, 2)
MOVE (5, 3) to (6, 1)
REGULAR SHELL at (6, 6)

45
MOVE (7, 3) to (5, 5)
BOULDER SHELL at (5, 1)
MOVE (3, 2) to (1, 2)
CLUSTER SHELL at (6, 2)
MOVE (5, 3) to (7, 3)
HEAL +1

45
MOVE (7, 3) to (5, 5)
BOULDER SHELL at (5, 1)
MOVE (3, 2) to (1, 2)
CLUSTER SHELL at (6, 2)
MOVE (5, 3) to (3, 2)
HEAL +1

45
MOVE (7, 3) to (5, 5)
BOULDER SHELL at (5, 1)
MOVE (3, 2) to (1, 2)
CLUSTER SHELL at (6, 2)
MOVE (5, 3) to (3, 2)
REGULAR SHELL at (3, 4)

45
MOVE (7, 3) to (5, 5)
BOULDER SHELL

BOULDER SHELL at (5, 1)
MOVE (3, 2) to (3, 4)
HEAL +1
MOVE (5, 3) to (7, 3)
HEAL +1

31
MOVE (7, 3) to (5, 5)
BOULDER SHELL at (5, 1)
MOVE (3, 2) to (3, 4)
HEAL +1
MOVE (5, 3) to (7, 3)
REGULAR SHELL at (7, 1)

31
MOVE (7, 3) to (5, 5)
BOULDER SHELL at (5, 1)
MOVE (3, 2) to (3, 4)
HEAL +1
MOVE (5, 3) to (3, 2)
HEAL +1

31
MOVE (7, 3) to (5, 5)
BOULDER SHELL at (5, 1)
MOVE (3, 2) to (3, 4)
HEAL +1
MOVE (5, 3) to (3, 2)
REGULAR SHELL at (6, 2)

31
MOVE (7, 3) to (5, 5)
BOULDER SHELL at (5, 1)
MOVE (3, 2) to (3, 4)
HEAL +1
MOVE (5, 3) to (3, 3)
HEAL +1

31
MOVE (7, 3) to (5, 5)
BOULDER SHELL at (5, 1)
MOVE (3, 2) to (3, 4)
HEAL +1
MOVE (5, 3) to (7, 4)
HEAL +1

31
MOVE (7, 3) to (5, 5)
BOULDER SHELL at (5, 1)
MOVE (3, 2) to (3, 4)
HEAL +1
MOVE (5, 3) to (7, 4)
REGULAR SHELL at (4, 4)

31
MOVE (7, 3) to (5, 5)
BOULDER SHELL at (5, 1)
MOVE (3, 2) to (3, 4)
HEAL +1
MOVE (5, 3) to (7, 4)
REGULAR SHELL at (7, 1)

31
MOVE (7, 3) to (5, 5)
BOULDER SHELL at (5, 1)
MOVE (3, 2) to (3, 4)
HEAL +1
MO

In [7]:
stop

NameError: name 'stop' is not defined

In [ ]:
ex = Executor()

In [ ]:
ex.execute(siege_mech.gen_actions()[2])

In [ ]:
grid.show()

In [ ]:
ex.execute(siege_mech.gen_actions()[4])

In [ ]:
grid.show()

In [ ]:
ex

In [ ]:
ex.undo()

In [ ]:
grid.show()

In [ ]:
stop

In [ ]:
ex = Executor()

score = DFS.rate(grid)

DFS.rate_dict(grid, True)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

ex.execute(frontier[1][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[1][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
print(score)

In [ ]:
ex.execute(frontier[1][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
print(score)

In [ ]:
ex.execute(frontier[2][0])
score = DFS.rate(grid)

show = grid.show()

dic = DFS.rate_dict(grid, True)

frontier = DFS.gen_rated_frontier(grid)
score = DFS.rate(grid)

frontier.sort(key=lambda x:x[1], reverse=True)

score = DFS.rate(grid)

ex.execute(frontier[0][0])
score = DFS.rate(grid)

dic = DFS.rate_dict(grid, True)

show = grid.show()

In [ ]:
print(score)

In [ ]:
ex.undo()
score = DFS.rate(grid)

show = grid.show()

ex.execute(frontier[1][0])
score = DFS.rate(grid)

grid.show()

ex.undo()
score = DFS.rate(grid)

grid.show()

ex.execute(frontier[2][0])
score = DFS.rate(grid)

grid.show()

grid.get_tile((3,6)).visitor

grid.get_tile((5,5)).visitor

grid.get_tile((4,6)).type_object.health

grid.get_tile((4,5)).type_object.health

DFS.rate_dict(grid, True)

grid.show()

In [ ]:
print(score)

new score 42 != old score 37 at 
MOVE (7, 3) to (5, 5)
MOVE (3, 2) to (3, 4)
CLUSTER SHELL at (3, 2)
BOULDER SHELL at (5, 1)
MOVE (5, 3) to (3, 3)
HEAL +1

[MOVE (7, 3) to (5, 5), MOVE (3, 2) to (3, 4), CLUSTER SHELL at (3, 2), BOULDER SHELL at (5, 1), MOVE (5, 3) to (7, 3), HEAL +1, 'UNDO', REGULAR SHELL at (7, 1), 'UNDO', 'UNDO', MOVE (5, 3) to (3, 2), HEAL +1, 'UNDO', REGULAR SHELL at (6, 2), 'UNDO', 'UNDO', MOVE (5, 3) to (3, 3), REGULAR SHELL at (3, 6), 'UNDO', REGULAR SHELL at (3, 7), 'UNDO', HEAL +1]

In [ ]:
stop

In [ ]:
stop

In [ ]:
dfs = DFS(grid, 5)

In [ ]:
dfs.show()

In [ ]:
stop

In [ ]:
grid.show()

In [ ]:
frontier = DFS.gen_rated_frontier(grid)

In [ ]:
frontier

In [ ]:
dfs = DFS(grid)

In [ ]:
DFS.rate_dict(grid)

In [ ]:
grid.show()

In [ ]:
stop

In [ ]:
[x for x in map(lambda x:x*5, [1,2,3,4,5,6])]

In [ ]:
grid.show()

In [ ]:
DFS.rate(grid)

In [ ]:
grid.show()

In [ ]:
DFS.gen_frontier(grid)

In [ ]:
def playback(code, grid):
    history = []
    for char in code:
        num = int(char)
        frontier = DFS.gen_frontier(grid)
        act = frontier[num]
        act.execute()
        history.append(act)
    return history
    

In [ ]:
grid.show()

In [ ]:
dfs = DFS(grid)

In [ ]:
ls = [(i,v) for i,v in dfs.explored.items()]
ls.sort(key=lambda iv: dfs.get_score_single(iv[1]), reverse=True)
ls    